In [1]:
from collections import defaultdict

In [2]:
import os
print(os.getcwd())

/workspace/HCAI_Project/main/data/passage


In [3]:
import pandas as pd
df = pd.read_csv('wsdm_rankagg_2013_readability_crowdflower_data.csv')
print(df.head())

    _unit_id      _created_at  _golden        _id _missed      _started_at  \
0  101706900  9/22/2011 10:18    False  213212730     NaN   9/22/2011 9:55   
1  101706900  9/22/2011 13:18    False  213286745     NaN  9/22/2011 13:15   
2  101706900  9/22/2011 13:42    False  213298361     NaN  9/22/2011 13:41   
3  101706900  9/22/2011 15:05    False  213338956     NaN  9/22/2011 15:03   
4  101706900  9/22/2011 16:39    False  213387820     NaN  9/22/2011 16:39   

   _tainted  _trust  _worker_id please_decide_which_passage_is_more_difficult  \
0      True  0.3333     1848539                  Passage B is more difficult.   
1     False  0.8571     1180070                  Passage A is more difficult.   
2     False  0.8571     3888076                  Passage A is more difficult.   
3      True  1.0000     3674807                  Passage B is more difficult.   
4      True  0.5000      756883                  Passage A is more difficult.   

   label_level_a  label_level_b  \
0        

In [4]:
df = df[df['label_level_a'] != df['label_level_b']]

In [5]:
for column, status in (
    ("passage_a", "Passage A is more difficult."),
    ("passage_b", "Passage B is more difficult."),
):
    df.loc[df["please_decide_which_passage_is_more_difficult"] == status, "label"] = (
        df.loc[df["please_decide_which_passage_is_more_difficult"] == status, column]
    )

In [6]:
df.rename(
    columns={"_worker_id": "performer", "passage_a": "left", "passage_b": "right"},
    inplace=True,
)

In [7]:
df.dropna(subset=["label"], inplace=True)

In [8]:
df[["performer", "left", "right", "label"]]

,performer,left,right,label
14,423,to find out how the adaptations help the polar...,houses and buildings. Did you know that animal...,to find out how the adaptations help the polar...
15,3927850,to find out how the adaptations help the polar...,houses and buildings. Did you know that animal...,to find out how the adaptations help the polar...
17,1068360,to find out how the adaptations help the polar...,houses and buildings. Did you know that animal...,to find out how the adaptations help the polar...
18,530218,to find out how the adaptations help the polar...,houses and buildings. Did you know that animal...,to find out how the adaptations help the polar...
19,2407979,to find out how the adaptations help the polar...,houses and buildings. Did you know that animal...,to find out how the adaptations help the polar...
...,...,...,...,...
13843,2803804,enough men sometimes into dry thwarted worms. ...,"see page in the booklet. Line Banks, savings a...",enough men sometimes into dry thwarted worms. ...
13844,1390511,enough men sometimes into dry thwarted worms. ...,"see page in the booklet. Line Banks, savings a...","see page in the booklet. Line Banks, savings a..."
13845,3481355,enough men sometimes into dry thwarted worms. ...,"see page in the booklet. Line Banks, savings a...","see page in the booklet. Line Banks, savings a..."
13846,3387099,enough men sometimes into dry thwarted worms. ...,"see page in the booklet. Line Banks, savings a...","see page in the booklet. Line Banks, savings a..."


In [9]:
def sort_df(df, column_name):
    # Sort by a specific column (replace 'column_name' with your column)
    df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False
    
    return df_sorted

In [10]:
gt_levels = {}

for _, row in df.iterrows():
    gt_levels[row["left"]] = int(row['label_level_a'])
    gt_levels[row["right"]] = int(row['label_level_b'])

gt = pd.Series(gt_levels.values(), index=gt_levels.keys())

In [11]:
gt_df = gt.reset_index()
gt_df.columns = ['label', 'score']
gt_df.tail()

,label,score
467,"Next, she puts on her red nose. It honks if yo...",1
468,"tears in her eyes. You look sad, said Sam. I a...",3
469,can really help you. The main idea is the main...,4
470,gave pencils to other students in the afternoo...,5
471,"come to the museum on Street, because no one e...",6


In [12]:
gt_df = sort_df(gt_df, 'score')

In [13]:
gt_df.tail()

,label,score
123,work over the summer on any changes they wish ...,12
113,between January and December plunged the Unite...,12
23,into a newly opened bank account. I was amazed...,12
11,"occurring phenomenon, manmade by products are ...",12
58,question requires students to use context clue...,12


In [14]:
gt_df.to_csv('gt_df_passage.csv')

In [15]:
item_labels = list(gt_df['label'])
item_label_dict = defaultdict(lambda:-1)
for i, item in enumerate(item_labels):
    item_label_dict[item] = i

In [16]:
len(item_label_dict)

472

In [17]:
import pandas as pd

# Assuming gt_df is your source DataFrame
df_passage = pd.DataFrame({
    'label': gt_df['label'],
    'score': gt_df['score']
})

# Optional: reset index if you want a clean sequential index
df_passage = df_passage.reset_index(drop=True)

print(df_passage.head())

                                               label  score
0  a star. Our planet, Earth, orbits, or circles,...      1
1  Adam, We did not have plastic toys. I played w...      1
2  Who said the little owl. Who wants to hunt wit...      1
3  dead leaf. This is a mole. Moles burrow underg...      1
4  ereaddatagradepsenvironcomp.html Environment r...      1


In [18]:
df_passage

,label,score
0,"a star. Our planet, Earth, orbits, or circles,...",1
1,"Adam, We did not have plastic toys. I played w...",1
2,Who said the little owl. Who wants to hunt wit...,1
3,dead leaf. This is a mole. Moles burrow underg...,1
4,ereaddatagradepsenvironcomp.html Environment r...,1
...,...,...
467,work over the summer on any changes they wish ...,12
468,between January and December plunged the Unite...,12
469,into a newly opened bank account. I was amazed...,12
470,"occurring phenomenon, manmade by products are ...",12


In [19]:
df = sort_df(df, 'performer')

In [20]:
# Get unique performers in the DataFrame
unique_performers = df['performer'].unique()

# Create a mapping from performer to a numeric label
performer_label_dict = defaultdict(lambda: -1)
for i, performer in enumerate(unique_performers):
    performer_label_dict[performer] = i

In [21]:
print(len(df))

11763


In [22]:
df.to_csv('passage_cleaned.csv')

In [21]:
PC_passage = {}
ans = 0
for performer, group in df.groupby('performer'):
    key = performer_label_dict[performer]  # map performer to key
    if key not in PC_passage:
        PC_passage[key] = list()  # initialize if not present

    for _, row in group.iterrows():

        left = row["left"].split("/")[-1]
        right = row["right"].split("/")[-1]
        winner = row["label"].split("/")[-1]  # whichever side was chosen

        # Map to indices
        left_label = item_label_dict[left]
        right_label = item_label_dict[right]
        winner_label = item_label_dict[winner]

        # Ensure pair stored as (winner, loser)
        if winner_label == left_label:
            PC_passage[key].append((left_label, right_label))
        else:
            PC_passage[key].append((right_label, left_label))
        
print(ans)

0


In [22]:
group_counts = df.groupby('performer').size()  # or .count() for column-wise counts
display(group_counts)
print(sum(group_counts))

performer
5          21
159         6
202         6
344         5
386        41
           ..
4450104     5
4452593     5
4452759     6
4453376    31
4455231     4
Length: 624, dtype: int64

11763


In [23]:
ans = 0
for i in PC_passage:
    ans += len(PC_passage[i])
print(ans)

11763


In [24]:
len(PC_passage)

624

In [25]:
PC_passage[0]

[(358, 286),
 (418, 400),
 (355, 460),
 (353, 188),
 (423, 89),
 (310, 16),
 (355, 270),
 (428, 404),
 (68, 459),
 (381, 8),
 (424, 87),
 (358, 175),
 (377, 24),
 (355, 207),
 (461, 386),
 (460, 417),
 (405, 85),
 (66, 433),
 (310, 111),
 (392, 364),
 (437, 426)]

In [28]:
import pickle
import os

# Make sure the directory exists
os.makedirs("data", exist_ok=True)

# Save PC_passage
with open("../PassagePC1.pickle", 'wb') as handle:
    pickle.dump(PC_passage, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save df_passage
df_passage.to_pickle("../PassageDF1.pickle")